Basic Neural Network Classifier for Modified MNist Dataset

In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

# Imports 
from __future__ import print_function
import os
import pickle as pkl
import cv2
import numpy as np
import pandas as pd
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils import data

# Import Data

In [2]:
og_trn_data = np.asarray(pd.read_pickle('train_max_x'))
num_samples = og_trn_data.shape[0]
num_features = og_trn_data.shape[1] * og_trn_data.shape[2]
og_trn_data = og_trn_data.flatten().reshape(num_samples, num_features)
print(og_trn_data.shape)

og_trn_labels = pd.read_csv('train_max_y.csv')['Label'].to_numpy()

og_test_data = np.asarray(pd.read_pickle('test_max_x'))
num_samples = og_test_data.shape[0]
num_features = og_test_data.shape[1] * og_test_data.shape[2]
og_test_data = og_test_data.flatten().reshape(num_samples, num_features)
print(og_test_data.shape)

(50000, 16384)
(10000, 16384)


# Dataset Class
Also includes a helper function to threshold the background out of the images.


In [3]:
def bwThresh(img):
  threshold = 200
  img[np.where(img < threshold)] = 0
  return img

class TrainingDataset(data.Dataset):
  'Characterizes the training dataset.'
  def __init__(self, transform=None):
    """
    Args:
        trn_file (string): Path to the zip file with training images.
        label_file (string): Path to the csv file with labels.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    # currently loading all data into memory
      # if that causes memory issues, will have 
      #   to load just a few point at a time
    self.trn_data = og_trn_data
    self.trn_labels = og_trn_labels
    self.transform = transform
  
  def __len__(self):
    'Denotes the total number of samples'
    return len(self.trn_labels)

  def __getitem__(self, index):
    'Generates one sample of data'
    # Select sample and its label
    sample = self.trn_data[index]
    label = self.trn_labels[index]

    if self.transform:
        sample = self.transform(sample)

    return sample, label

# Build Neural Network

From tutorial. Should be replaced!

In [4]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        
        Args:
            - D_in : input dimension of the data
            - H : size of the first hidden layer
            - D_out : size of the output/ second layer
        """
        super(TwoLayerNet, self).__init__() # intialize recursively 
        self.linear1 = torch.nn.Linear(D_in, H) # create a linear layer 
        self.linear2 = torch.nn.Linear(H, D_out) 

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data 
        and we must return a Variable of output data. We can use 
        Modules defined in the constructor as well as arbitrary 
        operators on Variables.
        """
        h_relu = self.linear1(x)
        y_pred = self.linear2(h_relu)
        return y_pred

# Instantiate the FNN
Most of the hyperparameters (except the threshold level of the images) are within this section, so tweak them accordingly.

In [9]:
# create dataset and train-validation split
dataset = TrainingDataset(transform=bwThresh)
train_dataset, val_dataset = torch.utils.data.dataset.random_split(dataset, [45000, 5000])

# create data loaders
params = {'batch_size': 64, 'shuffle': True, 'num_workers': 0}
train_loader = data.DataLoader(dataset=train_dataset, **params)
val_loader = data.DataLoader(dataset=val_dataset, **params)
full_trn_loader = data.DataLoader(dataset=dataset, **params)

# set size of neural network
D_in = 128 * 128   # equal to input shape (128x128 grayscale image)
H = 100            # TODO what should this be?
D_out = 10         # equal to output shape (class 0 to 9)

# instantiate neural network model
model = TwoLayerNet(D_in, H, D_out) # TODO change inner-layer dimensions
if torch.cuda.is_available():
    model.cuda()

# set loss criteria and optimizer of the neural network
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [10]:
# Training the FNN Model
for epoch in range(15):
    trn_losses = []
    val_losses = []

    # Training
    model.train()
    for x, y in train_loader:
        # Transfer to GPU if possible
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = criterion(yhat, y)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # save the loss
        trn_losses.append(loss.item())
  
    # Validation
    model.eval()
    with torch.no_grad():
        for x, y in val_loader:
            # Transfer to GPU
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()

            yhat = model(x)
            val_losses.append(criterion(yhat, y).item())

    print("Epoch {}: training loss {}, validation loss {}".format(epoch, sum(trn_losses) / len(trn_losses), sum(val_losses) / len(val_losses)))

torch.save(model.cpu(), "model.pkl")

Epoch 0: training loss 11.922115543349223, validation loss 7.2329354950144324
Epoch 1: training loss 3.6498066364702852, validation loss 4.033969058266169
Epoch 2: training loss 2.3334708555855532, validation loss 3.6025947166394583
Epoch 3: training loss 1.9152877911586652, validation loss 3.145274566698678
Epoch 4: training loss 1.7170990783382545, validation loss 3.204611880869805
Epoch 5: training loss 1.6013045575131069, validation loss 3.10971217819407
Epoch 6: training loss 1.527493071149696, validation loss 3.218778601175622
Epoch 7: training loss 1.4708385372703725, validation loss 3.0499460395378404
Epoch 8: training loss 1.422461903061379, validation loss 3.2023018674005437
Epoch 9: training loss 1.3822221806780859, validation loss 3.0982789842388296
Epoch 10: training loss 1.3462094454602762, validation loss 3.4529406692408307
Epoch 11: training loss 1.3191718748685988, validation loss 3.1445705015448073
Epoch 12: training loss 1.2877127538350495, validation loss 3.18638061

C:\Users\ahale\Anaconda3\envs\mcgill-venv\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TwoLayerNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ahale\Anaconda3\envs\mcgill-venv\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [11]:
model = model.cpu()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in full_trn_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 52 %
